# 基于MindSpore实现softmax

In [1]:
import sys
sys.path.append('..')

In [3]:
import mindspore
import numpy as np
from IPython import display

#batch_size = 256
#mnist_train, mnist_test = ms.load_data_mnist('../data', batch_size)

AttributeError: module 'cv2' has no attribute 'gapi_wip_gst_GStreamerPipeline'

In [ ]:

from mindvision.dataset import Mnist

# 下载并处理MNIST数据集
download_train = Mnist(path="./mnist", split="train", batch_size=256, repeat_num=1, shuffle=True, resize=32, download=True)

download_test = Mnist(path="./mnist", split="test", batch_size=256, resize=32, download=True)

mnist_train = download_train.run()
mnist_test = download_test.run()


将展平每个图像，把它们看作长度为784的向量。 因为我们的数据集有10个类别，所以网络输出维度为10

In [ ]:
num_inputs = 784
num_outputs = 10

给定一个矩阵X，我们可以对所有元素求和

In [ ]:
X = mindspore.Tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], mindspore.float32)
X.sum(0, keepdims=True), X.sum(1, keepdims=True)

实现**softmax**: $${softmax}(X)_{ij}=\frac{exp(X_{ij})}{\Sigma_{k}{exp(X_{ik})}}.$$


In [ ]:
import mindspore.ops as ops

def softmax(X):
    X_exp = ops.exp(X)
    partition = X_exp.sum(1, keepdims=True)
    return X_exp / partition

我们将每个元素变成一个非负数。此外，依据概率原理，每行总和为1

In [ ]:
X = mindspore.Tensor(np.random.normal(0, 1, (2, 5)), mindspore.float32)
X_prob = softmax(X)
X_prob, X_prob.sum(1)

In [ ]:
import mindspore.nn as nn
from mindspore import Tensor, Parameter
from mindspore.common.initializer import initializer, Zero, Normal

class Net(nn.Cell):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.W = Parameter(initializer(Normal(0.01, 0), (num_inputs, num_outputs), mindspore.float32))
        self.b = Parameter(initializer(Zero(), num_outputs, mindspore.float32))

    def construct(self, X):
        return softmax(ops.matmul(X.reshape((-1, self.W.shape[0])), self.W) + self.b)

创建一个数据样本y_hat，其中包含2个样本在3个类别的预测概率，及它们对应的标签y。 使用y作为y_hat中概率的索引

In [ ]:
y = mindspore.Tensor([0, 2], mindspore.int32)
y_hat = mindspore.Tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]], mindspore.float32)
y_hat[[0, 1], y]

实现交叉熵损失函数

In [ ]:
import mindspore.numpy as mnp

def cross_entropy(y_hat, y):
    return -mnp.log(y_hat[mnp.arange(y_hat.shape[0]), y])

cross_entropy(y_hat, y)

将预测类别与真实y元素进行比较

In [ ]:
def accuracy(y_hat, y):  
    """计算预测正确的数量。"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.asnumpy() == y.asnumpy()
    return float(cmp.sum())

accuracy(y_hat, y) / len(y)